In [1]:
# --- Imports ---

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

from tqdm import tqdm
import time
import math

In [2]:
# --- Configurations ---

class CONFIG:
    DATA_PATH = "./kaggle/input/nfl-big-data-bowl-2026-prediction/"

    N_FOLDS = 5
    EPOCHS = 10
    PATIENCE = 5
    FACTOR = .9
    LEARNING_RATE = 5e-4
    DROPOUT = 0.01
    FORCE_MAX = .8
    FORCE_MIN = .2


config = CONFIG()

In [3]:
# -- Dataset -- 

class TestDataset(Dataset):
    def __init__(self, ids, x0, x1a, x1b, x2, duration):
        # N: Number of training examples, L1: Length of input sequence, L2: Length of output sequence
        self.ids = ids                  # str list      # shape: N
        self.x0 = x0                    # np.arr(N)     # tensor(27)
        self.x1a = x1a                  # np.arr(N)     # np.arr(L1, 10)
        self.x1b = x1b                  # np.arr(N)     # np.arr(L1, 10)
        self.x2 = x2                    # np.arr(N)     # tensor(2)
        self.duration = duration        # int list      # shape: N

    def __len__(self):
        return len(self.x0)

    def __getitem__(self, idx):
        id = self.ids.iloc[idx]
        x0 = self.x0[idx]
        x1a = torch.tensor(self.x1a[idx], dtype=torch.float32)
        if self.x1b[idx] is None: x1b = torch.empty(0, x1a.shape[1], dtype=torch.float32)
        else: x1b = torch.tensor(self.x1b[idx], dtype=torch.float32)
        x2 = self.x2[idx]
        dur = self.duration[idx]
        return (id, x0, x1a, x1b, x2, dur)  

test_set = torch.load("test_set.pt", weights_only=False)

In [4]:
# -- Model --

class DotAttention(nn.Module):
    def forward(self, s_t, memory):
        s_t = s_t.unsqueeze(1)
        scores = torch.bmm(s_t, memory.transpose(1, 2)).squeeze(1)
        weights = F.softmax(scores, dim=1)
        context = torch.bmm(weights.unsqueeze(1), memory).squeeze(1)
        return context



class Encoder(nn.Module):
    def __init__(self, in_dim=10, hid_dim=256, n_layers=2):
        super().__init__()
        self.rnn = nn.LSTM(in_dim, hid_dim, n_layers, batch_first=True, dropout=CONFIG.DROPOUT)
        self.h0 = nn.Parameter(torch.randn(n_layers, 1, hid_dim))
        self.c0 = nn.Parameter(torch.randn(n_layers, 1, hid_dim))

    def forward(self, x1):
        if x1.size(1) > 0:
            e1, (h1, c1) = self.rnn(x1)
        else:
            e1 = torch.tensor([0])
            h1 = self.h0
            c1 = self.c0
        return  e1, h1, c1


class Decoder(nn.Module):
    def __init__(self, out_dim=2, hid_dim=256, n_layers=2):
        super().__init__()
        self.rnn = nn.LSTM(out_dim, hid_dim, n_layers, batch_first=True, dropout=CONFIG.DROPOUT)
        self.attn = DotAttention()
        self.out = nn.Sequential(
            nn.Linear(2*hid_dim, out_dim),
            nn.ReLU()
        )
    
    def forward(self, y0, h_c, mem):
        d0, h_c = self.rnn(y0, h_c)
        d0 = torch.squeeze(d0, dim=1)
        att = self.attn(d0, mem)
        d0 = torch.concat([d0, att], dim=1)
        d0 = self.out(d0)
        return d0, h_c
        

class MultiScaleCNNLSTMSeq2Seq(nn.Module):
    def __init__(self, in_dim=10, hid_dim=256, out_dim=2, const_dim=27):
        super().__init__()
        self.enc1 = Encoder()
        self.enc2 = Encoder()
        self.dec = Decoder()

        self.conv1 = nn.Sequential(
            nn.Conv1d(in_dim, hid_dim, 3, padding=1),
            nn.ReLU(),
            nn.Conv1d(hid_dim, hid_dim, 3, padding=1),
            nn.ReLU(),
            nn.Conv1d(hid_dim, hid_dim, 3, padding=1),
            nn.ReLU(),
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_dim, hid_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hid_dim, hid_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hid_dim, hid_dim, 5, padding=2),
            nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(in_dim, hid_dim, 7, padding=3),
            nn.ReLU(),
            nn.Conv1d(hid_dim, hid_dim, 7, padding=3),
            nn.ReLU(),
            nn.Conv1d(hid_dim, hid_dim, 7, padding=3),
            nn.ReLU(),
        )

        self.lin1 = nn.Linear(const_dim, hid_dim)
        self.lin2 = nn.Linear(const_dim, hid_dim)
        self.lin3 = nn.Linear(const_dim, hid_dim)
        self.lin4 = nn.Linear(const_dim, hid_dim)
        self.lin5 = nn.Linear(3*hid_dim, hid_dim)
        self.lin6 = nn.Linear(3*hid_dim, hid_dim)


    def forward(self, x0, x1a, x1b, x2, dur, force=False, trg=None):
        e1, h1, c1 = self.enc1(x1a)
        _, h2, c2 = self.enc2(x1b)

        x1a = torch.transpose(x1a, 1, 2)
        e2 = torch.transpose(self.conv1(x1a), 1, 2)
        e3 = torch.transpose(self.conv2(x1a), 1, 2)
        e4 = torch.transpose(self.conv3(x1a), 1, 2)

        mem = torch.concat([e1,e2,e3,e4], dim=1)
        y0 = torch.unsqueeze(x2, 1)

        h0 = torch.stack([self.lin1(x0), self.lin2(x0)], dim=0)
        h0 = torch.concat([h0, h1, h2], dim=2)
        h0 = F.relu(self.lin5(h0))      

        c0 = torch.stack([self.lin3(x0), self.lin4(x0)], dim=0)
        c0 = torch.concat([c0, c1, c2], dim=2)
        c0 = F.relu(self.lin6(c0))

        output = []

        for t in range(dur):
            d0, (h0, c0) = self.dec(y0, (h0,c0), mem)
            output.append(d0)
            if force:
                y0 = trg[:, t:(t+1), :]
            else:
                y0 = torch.unsqueeze(d0, dim=1)
            
        output = torch.stack(output).transpose(0,1)
        return output
            
        

In [ ]:
# -- Submission --

model = MultiScaleCNNLSTMSeq2Seq()
checkpoint = torch.load("MS-CNN-LSTM-Seq2Seq-1.pth", weights_only=True)
model.load_state_dict(checkpoint)
criterion = nn.MSELoss()
test_loader = DataLoader(
    test_set,
    batch_size=1,
    shuffle=True,
)

rows = []
with torch.no_grad():
    model.eval()
    for id, x0, x1a, x1b, x2, dur in test_loader:
        id = id[0]
        preds = model(x0, x1a, x1b, x2, dur)[0]
        
        for frame_num, (x, y) in enumerate(preds, start=1):
            rows.append({
                "id": f"{id}_{frame_num}",
                "x": x.item(),
                "y": y.item()
            })

submission_df = pd.DataFrame(rows)
submission_df.to_csv("submission.csv", index=False)

print("Submitted!!!")
submission_df.head()

Submitted!!!


,id,x,y
0,2024120805_3701_56540_1,13.007598,37.380402
1,2024120805_3701_56540_2,12.418032,37.615921
2,2024120805_3701_56540_3,12.494217,38.165253
3,2024120805_3701_56540_4,12.990677,38.053642
4,2024120805_3701_56540_5,13.263040,38.237686
